In [4]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [5]:
df = pd.read_csv("Data.csv")
df.dtypes

Direksi Perempuan      int64
Direksi Laki-Laki      int64
Proporsi Perempuan    object
Harga penutupan        int64
Total Shares           int64
Total Utang            int64
Total Asset            int64
dtype: object

In [6]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [7]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,155.000000,155.000000
mean,1.406452,0.509092
std,1.277664,2.036768
min,0.000000,0.007332
25%,0.500000,0.098036
50%,1.000000,0.144649
75%,2.000000,0.283211
max,6.000000,18.006205


In [8]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [9]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9692
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.326
Time:                        00:13:19   Log-Likelihood:                -329.21
No. Observations:                 155   AIC:                             662.4
Df Residuals:                     153   BIC:                             668.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6870      0.244      2.818      0.005       0.205       1.169
X             -0.1265      0.128     -0.984      0.326      -0.380       0.127
==============================================================================
Omnibus:                      266.429   Durbin-Watson:                   0.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30083.028
Skew:                           8.104   Prob(JB):                         0.00
Kurtosis:                      69.297   Cond. No.                         3.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [11]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     32.73
Date:                Tue, 16 May 2023   Prob (F-statistic):           5.41e-08
Time:                        00:13:19   Log-Likelihood:                -305.52
No. Observations:                 155   AIC:                             615.0
Df Residuals:                     153   BIC:                             621.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7907      0.209      3.779      0.000       0.377       1.204
X              0.6308      0.110      5.721      0.000       0.413       0.849
==============================================================================
Omnibus:                      272.977   Durbin-Watson:                   1.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33925.598
Skew:                           8.451   Prob(JB):                         0.00
Kurtosis:                      73.479   Cond. No.                         3.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [13]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [14]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [15]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [16]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [17]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [18]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [19]:
data_sample.describe()

,X,Y
count,135.000000,135.000000
mean,1.429630,0.166235
std,1.109933,0.107783
min,0.000000,0.007332
25%,1.000000,0.092929
50%,1.000000,0.133107
75%,2.000000,0.204609
max,4.000000,0.458297


In [20]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.120
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.148
Time:                        00:13:19   Log-Likelihood:                 110.74
No. Observations:                 135   AIC:                            -217.5
Df Residuals:                     133   BIC:                            -211.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1488      0.015      9.858      0.000       0.119       0.179
X              0.0122      0.008      1.456      0.148      -0.004       0.029
==============================================================================
Omnibus:                       23.028   Durbin-Watson:                   0.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.398
Skew:                           1.131   Prob(JB):                     4.13e-07
Kurtosis:                       3.333   Cond. No.                         3.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
bX = data_sample["X"] * result.params["X"]

In [22]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [23]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.144
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.145
Time:                        00:13:20   Log-Likelihood:                 174.54
No. Observations:                 135   AIC:                            -345.1
Df Residuals:                     133   BIC:                            -339.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0720      0.009      7.646      0.000       0.053       0.091
X              0.0076      0.005      1.464      0.145      -0.003       0.018
==============================================================================
Omnibus:                       26.849   Durbin-Watson:                   1.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.282
Skew:                           1.184   Prob(JB):                     2.18e-08
Kurtosis:                       3.817   Cond. No.                         3.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""